In [1]:
# Load text

In [2]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [3]:
in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner t


In [4]:
import string
import re
def clean_doc(doc):
    doc = doc.replace('--',' ')
    tokens = doc.split()
    re_punc = re.compile('[%s]'%re.escape(string.punctuation))
    tokens = [re_punc.sub('',w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [5]:
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' %len(tokens))
print('Unique Tokens: %d' %len(set(tokens)))

['i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid', 'us', 'wait'

In [6]:
# Save clean text

In [7]:
length = 50+1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' %len(sequences))

Total Sequences: 118631


In [8]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [10]:
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

In [11]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [12]:
# Encode sequences

In [13]:
from keras.preprocessing.text import Tokenizer

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [15]:
vocab_size = len(tokenizer.word_index) + 1

In [16]:
from numpy import array
from pickle import dump
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [18]:
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [17]:
def define_model(vocab_size, seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size,50,input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation="relu"))
    model.add(Dense(vocab_size, activation = "softmax"))
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    model.summary()
    plot_model(model, to_file="model.png", show_shapes=True)
    return model

In [19]:
model = define_model(vocab_size, seq_length)
model.fit(X,y, batch_size = 128, epochs = 10)

W0506 17:48:58.332640 140579629709120 deprecation.py:506] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1633: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            370500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________


W0506 17:48:59.916290 140579629709120 deprecation.py:323] From /home/arvind/.local/lib/python3.6/site-packages/keras/optimizers.py:550: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.
W0506 17:49:00.447460 140579629709120 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/10
118631/118631 [==============================] - 176s 1ms/step - loss: 6.1478 - accuracy: 0.0716
Epoch 2/10
118631/118631 [==============================] - 174s 1ms/step - loss: 5.6748 - accuracy: 0.1089
Epoch 3/10
118631/118631 [==============================] - 169s 1ms/step - loss: 5.4311 - accuracy: 0.1304
Epoch 4/10
118631/118631 [==============================] - 170s 1ms/step - loss: 5.2695 - accuracy: 0.1434
Epoch 5/10
118631/118631 [==============================] - 170s 1ms/step - loss: 5.1506 - accuracy: 0.1538
Epoch 6/10
118631/118631 [==============================] - 170s 1ms/step - loss: 5.0597 - accuracy: 0.1592
Epoch 7/10
118631/118631 [==============================] - 171s 1ms/step - loss: 4.9742 - accuracy: 0.1653
Epoch 8/10
118631/118631 [==============================] - 178s 1ms/step - loss: 4.8946 - accuracy: 0.1698
Epoch 9/10
118631/118631 [==============================] - 175s 1ms/step - loss: 4.8183 - accuracy: 0.1738
Epoch 10/10
118631/118631 [=

In [20]:
model.save('model.h5')
dump(tokenizer, open('tokenizer.pkl','wb'))

In [21]:
# Use language model

In [27]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [22]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [23]:
seq_length = len(lines[0].split()) - 1

In [28]:
model = load_model('model.h5')
tokenizer = load(open('tokenizer.pkl','rb'))

In [29]:
# Generate text

In [30]:
seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')

on the pegs of the instrument i might carry on the metaphor and speak after their manner of the blows which the plectrum gives and make accusations against the strings both of backwardness and forwardness to sound but this would be tedious and therefore i will only say that these are



In [31]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text  = seed_text
    for _  in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen = seq_length, truncating='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [32]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same and the same
